In [93]:
# python warning 방지용

import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [94]:
from elasticsearch import Elasticsearch

import yaml, json
import hashlib

class ElasticClient:
    
    def __init__(self, index_name: str, config_yaml_path: str, config_json_path: str):
        with open(config_yaml_path) as f:
            self.conf_yaml = yaml.safe_load(f)
            
        self.url = self.conf_yaml['elastic']['url']
        self.port = self.conf_yaml['elastic']['port']
        
        self.auth_id = self.conf_yaml['basic_auth']['id']
        self.auth_password = self.conf_yaml['basic_auth']['password']
        
        self.index = index_name
        
        self.es = Elasticsearch(f'{self.url}:{self.port}', basic_auth=(self.auth_id, self.auth_password), verify_certs=False)
    
        with open(config_json_path) as f:
            self.conf_json = json.load(f)
            
        self.settings = self.conf_json["settings"]
        self.mappings = self.conf_json['mappings']
        
        # 테이블 생성    
        if not self.es.indices.exists(index=self.index):
            self.es.indices.create(index=self.index, settings=self.settings, mappings=self.mappings)


    # 테이블 삭제
    def delete_index(self, index_name: str):
        if self.es.indices.exists(index= index_name):
            return self.es.indices.delete(index= index_name)


    # 도큐먼트 업데이트 > title은 불변 > content만 변경된다.
    # _id를 통해서 업데이트
    def update(self, document_id, data: dict):
        updata_body = {
            'doc': {
                'content': data['text']
            }
        }
        return self.es.update(index=self.index, id=document_id, body=updata_body)

    # 도큐먼트(피클 형식) - 삽입
    def upsert(self, file: dict):
        title, content = file['title'], file['text']
        
        data = {
            'title': title,
            'content': content
        }
        document_id = hashlib.md5(title.encode()).hexdigest()
        # title의 hash값을 _id로 설정
        # 동일한 title을 가진 document 삽입시 에러 발생 > 업데이트 진행
        # title을 그대로 _id로 설정할 경우 특수문자 등으로 인해서 에러 발생 가능성 존재하므로 안전하게 hash값으로 _id 설정
        try:
            result = self.es.index(index=self.index, body=data, id=document_id, timeout=None)
            return result
        except:
            update_result = self.updatae(document_id, data)
            return update_result


    # 해당 인덱스의 모든 도큐먼트 검색
    def search_all(self):
        body = {
            "query" : {
                "match_all": {}
            }
        }
        return self.es.search(index=self.index, body=body)


    # 쿼리에 따른 관련 도큐먼트 검색
    def search_by_field(self, query: str):
        body = {
            "query": {
                "multi_match": {
                    "query" : query,
                    "fields": ["content"],
                    "operator": "and"
                } 
            }
        }
        return self.es.search(index=self.index, body=body)
    
    
    # title의 hash 값이 _id에 해당함
    def search_by_id(self, title: str):
        document_id = hashlib.md5(title.encode()).hexdigest()
        
        body = {
            "query": {
                "term": {
                    "_id": document_id
                }
            }
        }
        
        return self.es.search(index=self.index, body=body)


    # 특정 필드와 일치하는 도큐먼트 삭제
    def delete_by_id(self, title: str):
        document_id = hashlib.md5(title.encode()).hexdigest()
        
        body = {
            "query": {
                "term": {
                    "_id": document_id
                }
            }
        }
        return self.es.delete_by_query(index=self.index, body=body)


    # 도큐먼트 일괄 삭제
    def delete_all(self):
        body = {
            "query": {
                "match_all": {}
            }
        }
        return self.es.delete_by_query(index=self.index, body=body)

In [95]:
es_model = ElasticClient("korean_analyzer", "../config.yaml", "../config.json")

/Users/ryan/.pyenv/versions/3.9.16/envs/namu/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [294]:
# es_model.delete_index("korean_analyzer")

ObjectApiResponse({'acknowledged': True})

In [133]:
# es_model.delete_all()

ObjectApiResponse({'took': 13, 'timed_out': False, 'total': 20, 'deleted': 20, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [13]:
import pickle
from tqdm import trange

for i in trange(0, 10):
    with open(f'../../db/namu_wiki_dump/{i}.pkl', 'rb') as f:
        namu_wiki_file = pickle.load(f)
        es_model.upsert(namu_wiki_file)

100%|██████████| 10/10 [00:00<00:00, 72.46it/s]


In [259]:
from pprint import pprint

pprint(es_model.search_all())

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 14, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'korean_analyzer', '_id': 'b869ccb498b58f56287dff0625aab2da', '_score': 1.0, '_source': {'title': '!!아앗!!', 'content': '！！ああっと！！\n▲신 세계수의 미궁 2에서 뜬 !!아앗!!\n세계수의 미궁 시리즈에 전통으로 등장하는 대사. 2편부터 등장했으며 훌륭한 사망 플래그의 예시이다.\n세계수의 모험가들이 탐험하는 던전인 수해의 구석구석에는 채취/벌채/채굴 포인트가 있으며, 이를 위한 채집 스킬에 투자하면 제한된 채집 기회에서 보다 큰 이득을 챙길 수 있다. 그러나 분배할 수 있는 스킬 포인트는 한정되어 있기 때문에 채집 스킬에 투자하는 만큼 전투 스킬 레벨은 낮아지게 된다. !!아앗!!이 발생하는 과정을 요약하면 다음과 같다.\n다만 채집 시스템은 신 세계수 시리즈의 그리모어 복제, 복합 채집 스킬인 야생의 감, 5편의 종족 특유 스킬, 크로스의 1레벨이 만렙인 채집 스킬 등으로 편의성이 점차 나아져서 채집 스킬 때문에 스킬 트리가 내려가는 일은 점점 줄어들었다.\n채집용 캐릭터들로 이루어진 약한 파티(ex: 레인저 5명)가 수해에 입장한다.\n필드 전투를 피해 채집 포인트에 도착한 후 열심히 아이템을 캐는 중에...\n!!아앗!!\n라플레시아가 나타났다!\n이때 등장하는 것은 FOE는 아니지만 훨씬 위층에 등장하는 강력한 필드 몬스터이며 선제 공격을 당하게 된다!\n으앙 죽음(hage)\n여담으로 !!아앗!!의 유래는 1인칭 던전 크롤러의 원조 위저드리에서 함정을 건드렸을 때 나오는 대사 Oops!(お

In [96]:
def elastic_explorer_filed(query: str):
    res = es_model.search_by_field(query)
    
    if len(res['hits']['hits']) == 0:
        print("검색결과가 없습니다. 다시 입력해주세요.")
        return 
    
    for idx, result in enumerate(res['hits']['hits']):
            pretty_out = f"{idx} - ID: {result['_id']}\nScore: {result['_score']}\nTitle: {result['_source']['title']}\n=Content=\n {result['_source']['content']}"
            print(pretty_out)
            
def elastic_explorer_id(query: str):
    res = es_model.search_by_id(query)
    
    if len(res['hits']['hits']) == 0:
        print("검색결과가 없습니다. 다시 입력해주세요.")
        return 
    
    for result in res['hits']['hits']:
            pretty_out = f"ID: {result['_id']}\nScore: {result['_score']}\nTitle: {result['_source']['title']}\n=Content=\n {result['_source']['content']}"
            print(pretty_out)

### upsert 확인

In [97]:
elastic_explorer_filed("에버랜드 창립일 알려줘")

0 - ID: ff62f5e36feaa20161bf629b822cca28
Score: 14.967764
Title: 제일모직
=Content=
 옛 삼성그룹(三星) 계열 패션, 의류, 섬유, 신소재 업체. 현 삼성물산 패션부문 및 롯데첨단소재의 뿌리이다. 약칭은 '일모'이다. 존속 당시 홈페이지 도메인은 'www.cii.samsung.co.kr'이었다.
창립 전 한국인들은 마카오에서 양복지를 사들여 양복을 해 입었는데, 문제는 수입 양복 값이 비쌌다. 이에 착안한 이병철 삼성물산 사장은 중역들의 만류에도 불구하고 1954년 9월 15일 제일모직공업을 세웠다. 1955년 소모방공장 준공 후 1956년 국산 양복지 '골덴텍스'를 만들었고, 이후 방적/직포/방모공장 등을 잇따라 세워 기틀을 다진 뒤, 1961년 창사 최초로 복지를 수출한 후 1968년 경북 경산공장을 세워 이듬해 학생복지 '에리트' 생산도 개시했으나, 1972년 7월 경산공장을 분리해 제일합섬(주)를 설립했다. 한때 제일제당, 제일합섬과 더불어 '제일' 삼총사 중 하나였다.
이후 제일제당은 계열 분리되어 CJ그룹이 되었고 제일합섬은 새한그룹으로 분리되었다가 외환위기 후 2000년에 해체되고, 웅진그룹에 인수되어 웅진케미칼이 되었다. 그리고 웅진그룹이 어려워지자 일본의 도레이가 인수하여 도레이케미칼이 되었다가 2019년 도레이첨단소재에 합병됐다.
1975년 기업공개 후 제일복장으로부터 '장미라사' 사업권을 이어받았고, 1976년 제일모직으로 상호를 변경했다. 같은 해 칠곡새마을공장을 필두로 1977년 안양디자인센터를 세우고 여성 기성복 '라보떼'를 출시했다. 1978년 부설여자중학교 및 실업고등학교를 열었고, 구미공장도 세운 뒤 탁구단도 창단했다. 1979년 기술연구소, 1981년 오스트레일리아 합작법인을 각각 냈고, 1982년 현지합작 양모공장을 세웠다. 1983년 안양 남성복생산공장 준공 후 1985년 신사복을 영국에 수출하며 1986년 카펫 생산을 개시했다. 1988년 의류사업본부(하티스트) 및

In [299]:
elastic_explorer_id("페텔기우스 로마네콩티")

ID: a66f68301704b5dfed39d826f8f7e598
Score: 1.0
Title: 페텔기우스 로마네콩티
=Content=
 アナタ、 怠惰ですね？
당신, 나태하군요?
脳が震えるううううううううううう！！！
뇌가 떨린다아아아아아아아아아아아!!!
ペテルギウス · ロマネコンティ| Petelgius romaneconti
라이트 노벨 Re: 제로부터 시작하는 이세계 생활의 등장인물. 성우는 width=20 마츠오카 요시츠구/ width=20 토드 하버콘
페텔기우스와 은근 썸을 타는 듯한
이미 이전부터 하이스쿨 D×D의 프리드 세르젠처럼 미치광이 캐릭터 연기를 제대로 소화해내는 광역계였지만 소드 아트 온라인의 키리토나 던전에서 만남을 추구하면 안 되는 걸까의 벨 크라넬처럼 성향이 비슷비슷한 주인공을 맡았던 탓에 국내에서 비슷비슷한 캐릭터를 맡는 성우, 특히 키리토 성우라는 인식이 강했으나 리제로가 한국에 알려지면서 팬들에게 자신의 연기력을 확실히 각인시키는 계기가 되었다.
그리고 포르투나의 성우는 이분이다. 노린 듯.
이 성우도 기존의 개구리 중사 케로로의 케로로, 페어리 테일의 나츠 드래그닐 등등 다양한 역할을 해낸 미국 성우다. 이전에 악역인 마기의 쥬다르와 소드 아트 온라인의 스고우 노부유키를 연기한 적이 있다.
마녀교 대죄주교 『나태』 담당. 나이 402세. 키 180cm. 몸무게 50kg 내외(최초일 때). 첫등장 당시 심녹색의 머리를 한 기괴한 모습을 하고 있었으며 밑에 후술할 원칠했던 과거 모습과는 완전히 다른데 아마 빙의 능력을 이용해 육체를 바꿨기 때문으로 보인다. 테마곡은 나태의 대죄주교.
저~는 마녀교 대죄주교,『나태』담당 ㅡ
'페텔기우스 로마네콩티'... 입니다!
☆Death☆⬆
이 "데스!"라고 외칠 때 음성이 매우 찰져서 니코동이나 티비플에서 인간관악기로 애용되는 중이다.(...) #
마녀교와 관련된 최초의 적이며 그 위협을 알림과 동시에 현재의 대죄주교중 능력이 최약수준임에도 최고급의 존재감을 드러낸 인물이며 그 행동들의 근간, 그와

In [300]:
print("= 업데이트 시작합니다.=")

file = {
        'title': "페텔기우스 로마네콩티",
        'text': "디지몬을 위해서"
        }

print("업데이트 내역: content가 '디지몬을 위해서'로 변경 중입니다.")

es_model.upsert(file)

print("= 업데이트 종료됩니다.=")

= 업데이트 시작합니다.=
업데이트 내역: content가 '디지몬을 위해서'로 변경 중입니다.
= 업데이트 종료됩니다.=


In [303]:
elastic_explorer_id('페텔기우스 로마네콩티')

ID: a66f68301704b5dfed39d826f8f7e598
Score: 1.0
Title: 페텔기우스 로마네콩티
=Content=
 디지몬을 위해서


In [201]:
es_model.delete_by_title("페텔기우스 로마네콩티")

ObjectApiResponse({'took': 11, 'timed_out': False, 'total': 1, 'deleted': 1, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [202]:
elastic_explorer("title","페텔기우스 로마네콩티")

In [203]:
elastic_explorer("title", "아앗")